In [ ]:
!pip install controlnet-aux==0.0.7
!pip install -U openmim
!pip install cog
!pip install mediapipe
!mim install mmengine
!mim install "mmcv>=2.0.1"
!mim install "mmdet>=3.1.0"
!mim install "mmpose>=1.1.0"

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#@title break video down into frames
import cv2

# Open the video file
cap = cv2.VideoCapture('/content/a.mp4')

i = 0
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == False:
        break

    # Save each frame of the video
    cv2.imwrite('/content/frames/frame_' + str(i) + '.jpg', frame)

    i += 1

cap.release()
cv2.destroyAllWindows()

In [ ]:
#==========Interpolate the pose frames==========
!pip install moviepy

In [ ]:
#@title interpolate processed frames (best to keep fps same as input video)
!ffmpeg -r 8 -i /content/test/frame_%d.jpg -c:v libx264 -vf "fps=8,format=yuv420p" testpose.mp4


In [ ]:
#=======AVAILABLE PROCESSORS========
# load processor from processor_id
# options are:
# ["canny", "depth_leres", "depth_leres++", "depth_midas", "depth_zoe", "lineart_anime",
#  "lineart_coarse", "lineart_realistic", "mediapipe_face", "mlsd", "normal_bae", "normal_midas",
#  "openpose", "openpose_face", "openpose_faceonly", "openpose_full", "openpose_hand",
#  "scribble_hed, "scribble_pidinet", "shuffle", "softedge_hed", "softedge_hedsafe",
#  "softedge_pidinet", "softedge_pidsafe", "dwpose"]

In [ ]:
#@title simply change the processor under 'processor' at the bottom, may need to add if not available, see above (goto v1 notebook if errors start)
#=======AVAILABLE PROCESSORS========
# load processor from processor_id
# options are:
# ["canny", "depth_leres", "depth_leres++", "depth_midas", "depth_zoe", "lineart_anime",
#  "lineart_coarse", "lineart_realistic", "mediapipe_face", "mlsd", "normal_bae", "normal_midas",
#  "openpose", "openpose_face", "openpose_faceonly", "openpose_full", "openpose_hand",
#  "scribble_hed, "scribble_pidinet", "shuffle", "softedge_hed", "softedge_hedsafe",
#  "softedge_pidinet", "softedge_pidsafe", "dwpose"]
import torch
import os
from typing import List
from cog import BasePredictor, Input, Path
from PIL import Image
from io import BytesIO
import time
from tqdm import tqdm
from controlnet_aux.processor import Processor
from controlnet_aux import (
    HEDdetector,
    MidasDetector,
    MLSDdetector,
    OpenposeDetector,
    PidiNetDetector,
    NormalBaeDetector,
    LineartDetector,
    LineartAnimeDetector,
    CannyDetector,
    ContentShuffleDetector,
    ZoeDetector,
    MediapipeFaceDetector,
    SamDetector,
    LeresDetector,
    DWposeDetector,
)

#Processor = processor
image_dir = '/content/frames'

class Predictor(BasePredictor):
    def setup(self) -> None:
        """Load the model into memory to make running multiple predictions efficient"""

        self.annotators = {
            "canny": CannyDetector(),
            "content": ContentShuffleDetector(),
            "face_detector": MediapipeFaceDetector(),
            "hed": self.initialize_detector(HEDdetector),
            "midas": self.initialize_detector(MidasDetector),
            "mlsd": self.initialize_detector(MLSDdetector),
            "open_pose": self.initialize_detector(OpenposeDetector),
            "pidi": self.initialize_detector(PidiNetDetector),
            "normal_bae": self.initialize_detector(NormalBaeDetector),
            "lineart": self.initialize_detector(LineartDetector),
            "lineart_anime": self.initialize_detector(LineartAnimeDetector),
            # "zoe": self.initialize_detector(ZoeDetector),


            # "mobile_sam": self.initialize_detector(
            #     SamDetector,
            #     model_name="dhkim2810/MobileSAM",
            #     model_type="vit_t",
            #     filename="mobile_sam.pt",
            # ),
            "leres": self.initialize_detector(LeresDetector),
        }

        torch.device("cuda")

    def initialize_detector(
        self, detector_class, model_name="lllyasviel/Annotators", **kwargs
    ):
        return detector_class.from_pretrained(
            model_name,
            cache_dir="model_cache",
            **kwargs,
        )

    def process_images(self, image_dir: str) -> List[Path]:
    # Start time for overall processing
     start_time = time.time()

    # Load all images into memory
    images = [Image.open(os.path.join(image_dir, image_name)).convert("RGB").resize((512, 512)) for image_name in os.listdir(image_dir)]

    paths = []

    def predict(
    self,
    image_dir: str = Input(
        default="/content/frames",
        description="Directory containing the images to be processed"
    )
):

        canny: bool = Input(
            default=True,
            description="Run canny edge detection",
        ),
        content: bool = Input(
            default=True,
            description="Run content shuffle detection",
        ),
        face_detector: bool = Input(
            default=True,
            description="Run face detection",
        ),
        hed: bool = Input(
            default=True,
            description="Run HED detection",
        ),
        midas: bool = Input(
            default=True,
            description="Run Midas detection",
        ),
        mlsd: bool = Input(
            default=True,
            description="Run MLSD detection",
        ),
        open_pose: bool = Input(
            default=True,
            description="Run Openpose detection",
        ),
        pidi: bool = Input(
            default=True,
            description="Run PidiNet detection",
        ),
        normal_bae: bool = Input(
            default=True,
            description="Run NormalBae detection",
        ),
        lineart: bool = Input(
            default=True,
            description="Run Lineart detection",
        ),
        lineart_anime: bool = Input(
            default=True,
            description="Run LineartAnime detection",

        ),
        leres: bool = Input(
            default=True,
            description="Run Leres detection",
        ),


        # Load image
        # Load all images into memory
        start_time = time.time()  # Start time for overall processing
        images = [Image.open(os.path.join(image_dir, image_name)).convert("RGB").resize((512, 512)) for image_name in os.listdir(image_dir)]

        paths = []
        annotator_inputs = {
            "canny": canny,
            "content": content,
            "face_detector": face_detector,
            "hed": hed,
            "midas": midas,
            "mlsd": mlsd,
            "open_pose": open_pose,
            "pidi": pidi,
            "normal_bae": normal_bae,
            "lineart": lineart,
            "lineart_anime": lineart_anime, "openpose_full": openpose_full,

            "leres": leres,
        }
        for annotator, run_annotator in annotator_inputs.items():
            if run_annotator:
                processed_image = self.process_image(image, annotator)
                #processed_image.save(f"/tmp/{annotator}.png")
                processed_path = f'/content/test1/{image_name}'

        return paths

import time
from tqdm import tqdm

# Load images and paths
images = []
image_paths = []
for name in os.listdir(image_dir):
  path = os.path.join(image_dir, name)
  image = Image.open(path)

  images.append(image)
  image_paths.append(path)

# Process images
processed = [
  Processor("openpose_full") for path in tqdm(image_paths)
]

# Save processed
import os
from PIL import Image

# Get a list of filenames in /content/frames
filenames = os.listdir('/content/frames')

# Save processed
for filename in filenames:

  # Get the full path of the image file
  image_path = os.path.join('/content/frames', filename)

  # Load the image
  image = Image.open(image_path)

  # Process image
  # Process all images with progress bar
  processed_image = processor(image, to_pil=True)

  # Extract original name
  original_name = filename.split('.')[0]

  # Save image
  processed_path = f'/content/test2/{original_name}.png'
  processed_image.save(processed_path)



In [ ]:
@#title for seeing whats in controlnet_aux
dir(controlnet_aux)

In [ ]:
!zip -r nameof.zip <location of files and folder>

In [ ]:
#@title Login to HuggingFace 🤗

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
# https://huggingface.co/settings/tokens
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

In [ ]:
@#title upload to Huggingface
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="",
    path_in_repo="name.zip",
    repo_id="",
    repo_type="dataset",
)